In [1]:
import calendar
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
#from pandas.tseries.offsets import BDay
from pandas.tseries.offsets import * 

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.options.display.max_rows = 11
pd.options.display.float_format = '{:.2f}'.format

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 8, 23), datetime.date(2023, 8, 22))

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
to_mmdd = today.strftime('%m-%d')
print(f'Today in mm-dd format: {to_mmdd}')
print(f'yesterday: {yesterday}')

today: 2023-08-23
Today in mm-dd format: 08-23
yesterday: 2023-08-22 00:00:00


In [3]:
one_month_ago = yesterday + DateOffset(days = -28)
print('One month ago: ', one_month_ago, ' ', one_month_ago.day_name())

One month ago:  2023-07-25 00:00:00   Tuesday


In [4]:
format_dict = {
    'ttl_qty':'{:,}','qty':'{:,}',
    'from_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}','fm_price':'{:.2f}',
    'price':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','pct':'{:,.2f}%',  
}

cols = 'name fm_md to_md days fm_price to_price pct spd avg max_price min_price ttl_qty trend'.split()

### Get past one quarter data

In [5]:
sql = """
SELECT name
FROM buy 
ORDER BY name
"""
buy = pd.read_sql(sql, const)

names = buy['name'].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'AH', 'AIMIRT', 'ASK', 'ASP', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART'"

In [6]:
in_p = in_p + ", 'BEM', 'EA'"
in_p

"'AH', 'AIMIRT', 'ASK', 'ASP', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART', 'BEM', 'EA'"

### Bypass this statement if aboce statement is executed

In [7]:
sql = """
SELECT name
FROM orders
ORDER BY trade, name
"""
#buy = pd.read_sql(sql, conlite)

#names = buy['name'].values.tolist()
#in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'AH', 'AIMIRT', 'ASK', 'ASP', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART', 'BEM', 'EA'"

### End of Selection between Buy or Orders

In [8]:
sql = """
SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (one_month_ago, in_p)
print(sql)


SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '2023-07-25 00:00:00' AND name IN ('AH', 'AIMIRT', 'ASK', 'ASP', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART', 'BEM', 'EA') 
ORDER BY name, date


In [9]:
price = pd.read_sql(sql, const)
price.sample(5)

,name,date,price,qty,maxp,minp
131,BEM,2023-08-22,9.00,24169458,9.00,8.90
572,WHART,2023-07-27,10.60,445461,10.70,10.50
534,TMT,2023-07-27,7.25,96525,7.30,7.20
478,STA,2023-07-31,16.30,6680581,16.60,16.10
539,TMT,2023-08-07,7.20,45439,7.30,7.20


In [10]:
data_path = "../data/"
file_name = "Quarterly-Price-by-Name.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

price.to_csv(output_file, header=None, index=False)
price.to_csv(osd_file, header=True, index=False)

### Call ruby ruby\crt-summary.rb

In [11]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [12]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [13]:
!ruby ruby\\crt-summary.rb

ruby/../Data/Quarterly-Price-by-Name.csv
name|fm_date|to_date|days|fm_price|to_price|diff|pct|spd|avg|max_price|min_price|trend|nbr|ttl_qty
AH|2023-07-25|2023-07-25|1|33.25|33.25|0.0|0.0|0|0|33.25|32.25||2|844,875
AH|2023-07-26|2023-07-26|1|32.5|32.5|0.0|0.0|0|0|33.25|32.5|Downward|3|708,301
AH|2023-07-27|2023-07-31|3|32.75|33.25|0.5|1.53|2|0|33.25|32.25|Upward|5|1,959,945
AH|2023-08-02|2023-08-03|2|32.25|31.75|-0.5|-1.55|-2|-1|33.25|31.5|Downward|7|2,672,397
AH|2023-08-04|2023-08-04|1|32.75|32.75|0.0|0.0|0|0|33.25|31.75|Upward|8|801,428
AH|2023-08-07|2023-08-08|2|32.5|32.25|-0.25|-0.77|-1|-1|33.0|32.25|Downward|10|804,960
AH|2023-08-09|2023-08-11|3|32.5|33.0|0.5|1.54|2|0|33.0|32.0|Upward|13|1,408,815
AH|2023-08-15|2023-08-16|2|32.0|32.0|0.0|0.0|0|0|32.75|31.75|Downward|15|2,077,700
AH|2023-08-17|2023-08-18|2|33.25|34.5|1.25|3.76|5|2|34.75|32.25|Upward|17|3,655,135
AH|2023-08-21|2023-08-23|3|34.0|34.0|0.0|0.0|0|0|34.75|33.75|Downward|20|2,201,115
AIMIRT|2023-07-25|2023-07-31|5|11.0|10.

In [14]:
os.chdir("C:\\users\\user\\onedrive\\a7\\daily")
%pwd


SENA|2023-08-11|2023-08-18|6|2.98|2.76|-0.22|-7.38|-11|-2|3.0|2.76|Downward|454|5,429,436
SENA|2023-08-21|2023-08-23|3|2.78|2.82|0.04|1.44|2|0|2.82|2.74|Upward|457|1,489,636
SINGER|2023-07-25|2023-07-25|1|8.7|8.7|0.0|0.0|0|0|8.9|8.65|Upward|458|3,610,362
SINGER|2023-07-26|2023-07-26|1|8.1|8.1|0.0|0.0|0|0|8.8|8.05|Downward|459|11,408,182
SINGER|2023-07-27|2023-07-27|1|8.2|8.2|0.0|0.0|0|0|8.4|8.05|Upward|460|18,502,840
SINGER|2023-07-31|2023-08-08|7|8.0|7.35|-0.65|-8.13|-13|-2|8.25|7.35|Downward|466|41,979,546
SINGER|2023-08-09|2023-08-09|1|7.55|7.55|0.0|0.0|0|0|7.6|7.2|Upward|467|7,469,266
SINGER|2023-08-10|2023-08-10|1|7.45|7.45|0.0|0.0|0|0|7.65|7.4|Downward|468|4,964,418
SINGER|2023-08-11|2023-08-11|1|8.6|8.6|0.0|0.0|0|0|8.65|7.2|Upward|469|74,945,046
SINGER|2023-08-15|2023-08-16|2|8.5|8.2|-0.3|-3.53|-6|-3|8.9|8.2|Downward|471|55,040,296
SINGER|2023-08-17|2023-08-18|2|9.7|9.85|0.15|1.55|3|1|9.9|8.05|Upward|473|99,886,808
SINGER|2023-08-21|2023-08-22|2|10.2|10.2|0.0|0.0|0|0|10.8|10.1|

'C:\\users\\user\\onedrive\\a7\\daily'

TFFIF|2023-08-03|2023-08-08|4|7.55|7.55|0.0|0.0|0|0|7.55|7.45|Upward|523|6,593,392
TFFIF|2023-08-09|2023-08-15|5|7.5|7.5|0.0|0.0|0|0|7.55|7.5|Downward|527|6,473,203
TFFIF|2023-08-16|2023-08-18|3|7.55|7.6|0.05|0.66|1|0|7.6|7.55|Upward|530|3,806,804
TFFIF|2023-08-21|2023-08-23|3|7.55|7.45|-0.1|-1.32|-2|-1|7.6|7.45|Downward|533|7,656,376
TMT|2023-07-25|2023-07-25|1|7.2|7.2|0.0|0.0|0|0|7.25|7.2|Downward|534|102,569
TMT|2023-07-26|2023-07-31|4|7.25|7.25|0.0|0.0|0|0|7.3|7.15|Upward|537|161,459
TMT|2023-08-02|2023-08-08|5|7.2|7.15|-0.05|-0.69|-1|-1|7.3|7.15|Downward|542|216,416
TMT|2023-08-09|2023-08-10|2|7.25|7.25|0.0|0.0|0|0|7.25|7.15|Upward|544|33,856
TMT|2023-08-11|2023-08-17|5|7.2|7.0|-0.2|-2.78|-4|-1|7.25|6.95|Downward|548|660,755
TMT|2023-08-18|2023-08-21|2|7.1|7.1|0.0|0.0|0|0|7.2|7.0|Upward|550|100,525
TMT|2023-08-22|2023-08-22|1|7.05|7.05|0.0|0.0|0|0|7.15|7.05|Downward|551|101,727
TMT|2023-08-23|2023-08-23|1|7.2|7.2|0.0|0.0|0|0|7.2|7.0|Upward|552|115,020
WHAIR|2023-07-25|2023-08-03|8

In [15]:
file_name   = 'summary.csv'
input_file = data_path + file_name
df = pd.read_csv(input_file, sep=',', index_col=None)
df.sample(5).style.format(format_dict)

,name,from_date,to_date,days,from_price,to_price,diff,pct,spd,avg,max_price,min_price,ttl_qty,trend,nbr
186,RCL,2023-08-15,2023-08-16,2,22.20,21.20,-1.00,-4.50%,-10,-5,23.90,21.20,"16,066,980",Downward,414
69,DIF,2023-08-10,2023-08-10,1,10.00,10.00,0.00,0.00%,0,0,10.00,9.90,"11,787,458",Downward,164
9,AH,2023-08-21,2023-08-23,3,34.00,34.00,0.00,0.00%,0,0,34.75,33.75,"2,201,115",Downward,20
127,KCE,2023-08-04,2023-08-07,2,42.75,42.75,0.00,0.00%,0,0,43.75,41.50,"28,120,197",Upward,294
243,TMT,2023-08-11,2023-08-17,5,7.20,7.00,-0.20,-2.78%,-4,-1,7.25,6.95,"660,755",Downward,548


In [16]:
df.rename(columns={'from_date':'fm_date','from_price':'fm_price'},inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df['fm_md'] = df['fm_date'].dt.strftime('%m-%d')
df['to_md'] = df['to_date'].dt.strftime('%m-%d')

### Group analysis

In [17]:
mask_p = df['to_md'] == to_mmdd
df[mask_p][cols].sort_values('pct',ascending=False).head().style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
232,SYNEX,08-21,08-23,3,11.70,12.50,6.84%,8,2,13.00,11.60,"8,006,188",Upward
115,JMART,08-21,08-23,3,20.00,21.30,6.50%,13,4,21.70,19.60,"123,587,389",Upward
124,JMT,08-21,08-23,3,43.00,45.75,6.40%,11,3,46.50,42.50,"48,753,503",Upward
25,ASP,07-25,08-23,22,2.80,2.96,5.71%,8,0,2.96,2.80,"15,049,010",Upward
79,EA,08-21,08-23,3,58.75,61.00,3.83%,9,3,61.25,57.25,"60,760,736",Upward


In [18]:
name_up = 'CPNREIT'
mask_n = df.name == name_up
df[mask_n][cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
55,CPNREIT,07-25,08-02,7,10.60,11.00,3.77%,4,0,12.30,10.20,"37,185,881",Upward
56,CPNREIT,08-03,08-03,1,10.90,10.90,0.00%,0,0,11.30,10.90,"1,116,658",Downward
57,CPNREIT,08-04,08-07,2,11.10,11.10,0.00%,0,0,11.20,10.90,"1,402,862",Upward
58,CPNREIT,08-08,08-10,3,10.90,10.90,0.00%,0,0,11.10,10.80,"2,352,804",Downward
59,CPNREIT,08-11,08-15,3,11.10,11.10,0.00%,0,0,11.20,10.80,"3,201,172",Upward
60,CPNREIT,08-16,08-23,6,11.00,10.90,-0.91%,-1,-1,11.10,10.60,"10,609,149",Downward


In [19]:
mask_m = df['to_md'] == to_mmdd
df[mask_m][cols].sort_values('pct',ascending=True).head().style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
71,DIF,08-15,08-23,7,9.75,9.15,-6.15%,-12,-2,9.75,9.10,"136,438,825",Downward
96,IVL,08-18,08-23,4,30.50,29.00,-4.92%,-6,-2,31.00,28.50,"64,685,012",Downward
14,AIMIRT,08-22,08-23,2,11.10,10.90,-1.80%,-2,-1,11.20,10.80,"523,100",Downward
238,TFFIF,08-21,08-23,3,7.55,7.45,-1.32%,-2,-1,7.60,7.45,"7,656,376",Downward
146,NER,08-22,08-23,2,4.62,4.56,-1.30%,-3,-2,4.66,4.54,"14,648,742",Downward


In [20]:
name_down = 'PTG'
mask_n = df.name == name_down
df[mask_n][cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
159,PTG,07-25,07-25,1,10.60,10.60,0.00%,0,0,10.60,10.40,"4,549,759",Upward
160,PTG,07-26,07-26,1,10.50,10.50,0.00%,0,0,10.60,10.40,"4,715,696",Downward
161,PTG,07-27,07-31,3,10.70,10.80,0.93%,1,0,10.80,10.30,"15,992,429",Upward
162,PTG,08-02,08-08,5,10.50,10.10,-3.81%,-4,-1,10.80,10.00,"50,621,416",Downward
163,PTG,08-09,08-10,2,10.30,10.40,0.97%,1,0,10.50,10.00,"8,893,651",Upward
164,PTG,08-11,08-16,4,10.20,10.00,-1.96%,-2,-1,10.30,10.00,"20,995,650",Downward
165,PTG,08-17,08-17,1,10.30,10.30,0.00%,0,0,10.50,9.95,"9,561,125",Upward
166,PTG,08-18,08-18,1,10.10,10.10,0.00%,0,0,10.40,10.00,"9,598,838",Downward
167,PTG,08-21,08-22,2,10.30,10.70,3.88%,4,2,10.70,10.00,"19,911,990",Upward
168,PTG,08-23,08-23,1,10.60,10.60,0.00%,0,0,10.70,10.50,"7,628,851",Downward


In [21]:
df_groupby_trend = df.groupby('trend')

In [22]:
# To retrieve one of the created groups
df_up = df_groupby_trend.get_group('Upward')
df_up[cols].sample(5).style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
167,PTG,08-21,08-22,2,10.30,10.70,3.88%,4,2,10.70,10.00,"19,911,990",Upward
46,BEM,07-27,07-31,3,9.00,9.10,1.11%,2,0,9.10,8.90,"58,408,789",Upward
137,NER,07-25,07-25,1,4.62,4.62,0.00%,0,0,4.66,4.56,"7,315,380",Upward
176,PTTGC,08-17,08-17,1,37.25,37.25,0.00%,0,0,37.25,36.00,"13,665,412",Upward
8,AH,08-17,08-18,2,33.25,34.50,3.76%,5,2,34.75,32.25,"3,655,135",Upward


In [23]:
# To retrieve one of the created groups
df_dw = df_groupby_trend.get_group('Downward')
df_dw[cols].sample(5).style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
130,KCE,08-18,08-21,2,46.50,46.50,0.00%,0,0,47.50,45.75,"15,579,244",Downward
112,JMART,08-16,08-16,1,18.10,18.10,0.00%,0,0,18.70,17.70,"44,164,520",Downward
72,EA,07-25,07-26,2,55.75,54.75,-1.79%,-4,-2,56.25,54.25,"16,967,955",Downward
9,AH,08-21,08-23,3,34.00,34.00,0.00%,0,0,34.75,33.75,"2,201,115",Downward
195,SCC,08-18,08-22,3,314.00,313.00,-0.32%,-1,-1,318.00,311.00,"5,563,716",Downward


In [24]:
# there is a function called .agg() and it allows us to specify multiple aggregation functions at once
df.groupby('trend').days.agg(['max', 'count', 'median', 'mean'])

,max,count,median,mean
trend,,,,
Downward,12,130,2.00,2.49
Upward,22,133,2.00,2.38


In [25]:
upward_max = df.groupby('trend').days.max().loc['Upward']
downward_max = df.groupby('trend').days.max().loc['Downward']

In [26]:
df.query('days == @upward_max and trend == "Upward"')[cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
25,ASP,07-25,08-23,22,2.80,2.96,5.71%,8,0,2.96,2.80,"15,049,010",Upward


In [27]:
df.query('days == @downward_max and trend == "Downward"')[cols]

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
133,MCS,07-26,08-10,12,6.55,6.10,-6.87,-9,-1,6.60,5.90,3361083,Downward


### Individual stock analysis

In [28]:
df.query('name == @name_up')[cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
55,CPNREIT,07-25,08-02,7,10.60,11.00,3.77%,4,0,12.30,10.20,"37,185,881",Upward
56,CPNREIT,08-03,08-03,1,10.90,10.90,0.00%,0,0,11.30,10.90,"1,116,658",Downward
57,CPNREIT,08-04,08-07,2,11.10,11.10,0.00%,0,0,11.20,10.90,"1,402,862",Upward
58,CPNREIT,08-08,08-10,3,10.90,10.90,0.00%,0,0,11.10,10.80,"2,352,804",Downward
59,CPNREIT,08-11,08-15,3,11.10,11.10,0.00%,0,0,11.20,10.80,"3,201,172",Upward
60,CPNREIT,08-16,08-23,6,11.00,10.90,-0.91%,-1,-1,11.10,10.60,"10,609,149",Downward


In [29]:
df_tmp = price.query('name == @name_up')
df_tmp.describe().round(2)

,price,qty,maxp,minp
count,19.00,19.00,19.00,19.00
mean,10.94,2940448.74,11.10,10.75
std,0.14,3955333.69,0.32,0.28
min,10.60,324504.00,10.80,9.90
25%,10.90,884068.00,11.00,10.70
50%,11.00,1405208.00,11.00,10.90
75%,11.00,2079599.00,11.15,10.90
max,11.10,14386299.00,12.30,11.00


In [30]:
df.query('name == @name_down')[cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
159,PTG,07-25,07-25,1,10.60,10.60,0.00%,0,0,10.60,10.40,"4,549,759",Upward
160,PTG,07-26,07-26,1,10.50,10.50,0.00%,0,0,10.60,10.40,"4,715,696",Downward
161,PTG,07-27,07-31,3,10.70,10.80,0.93%,1,0,10.80,10.30,"15,992,429",Upward
162,PTG,08-02,08-08,5,10.50,10.10,-3.81%,-4,-1,10.80,10.00,"50,621,416",Downward
163,PTG,08-09,08-10,2,10.30,10.40,0.97%,1,0,10.50,10.00,"8,893,651",Upward
164,PTG,08-11,08-16,4,10.20,10.00,-1.96%,-2,-1,10.30,10.00,"20,995,650",Downward
165,PTG,08-17,08-17,1,10.30,10.30,0.00%,0,0,10.50,9.95,"9,561,125",Upward
166,PTG,08-18,08-18,1,10.10,10.10,0.00%,0,0,10.40,10.00,"9,598,838",Downward
167,PTG,08-21,08-22,2,10.30,10.70,3.88%,4,2,10.70,10.00,"19,911,990",Upward
168,PTG,08-23,08-23,1,10.60,10.60,0.00%,0,0,10.70,10.50,"7,628,851",Downward


In [31]:
df_tmp = price.query('name == @name_down')
df_tmp.describe().round(2)

,price,qty,maxp,minp
count,19.00,19.00,19.00,19.00
mean,10.37,8024705.53,10.48,10.17
std,0.23,4015100.35,0.20,0.21
min,10.00,2925694.00,10.10,9.90
25%,10.20,4632727.50,10.30,10.00
50%,10.30,7032523.00,10.50,10.20
75%,10.55,10526087.50,10.65,10.35
max,10.80,17325718.00,10.80,10.60


In [32]:
mask_p = df['to_md'] == to_mmdd
summary = df[mask_p][cols].sort_values(['pct','trend'],ascending=[False,False])

In [33]:
file_name = 'summary-240.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

summary[cols].to_csv(output_file, header=True, index=False)
summary[cols].to_csv(data_file, header=True, index=False)
summary[cols].to_csv(one_file, header=True, index=False)
summary[cols].to_csv(osd_file, header=True, index=False)

In [34]:
sql = """
SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '%s' 
ORDER BY name, date"""
sql = sql % (today)
print(sql)


SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '2023-08-23' 
ORDER BY name, date


In [35]:
df = pd.read_sql(sql, const)
df

,name,date,price,qty,maxp,minp
0,ACE,2023-08-23,1.76,6690748,1.78,1.73
1,ADVANC,2023-08-23,222.00,4643395,223.00,220.00
2,AEONTS,2023-08-23,172.00,574632,172.00,167.00
3,AH,2023-08-23,34.00,933715,34.75,34.00
4,AIMIRT,2023-08-23,10.90,330600,11.00,10.80
...,...,...,...,...,...,...
185,WHAIR,2023-08-23,6.75,98849,6.85,6.70
186,WHART,2023-08-23,10.50,789544,10.60,10.40
187,WHAUP,2023-08-23,3.92,3510621,3.92,3.82
188,WICE,2023-08-23,7.70,792421,7.70,7.60
